llmware\examples\llm_prompts.py


```
This example demonstrates:
      1. Prompting LLMs with different kinds of sources/context
      2. The Prompt Catalog and the use different prompt styles

      Note: This example uses OpenAI's gpt-4 LLM.
```

In [1]:
import os
from llmware.prompts import Prompt
from llmware.setup import Setup
from llmware.util import PromptCatalog

In [2]:
OPENAI_MODEL_NAME = "gpt-3.5-turbo" # "gpt-4"

In [3]:
# Update this value with your own API Key, either by setting the env var or editing it directly here:
openai_api_key = os.environ.get("OPENAI_API_KEY","")

In [5]:
# llmware provides many out of the box prompt instructions such as yes_no, number_or_none, summarize_with_bullets,etc
def print_all_prompt_instructions():
    print (f"\n > ALL AVAILABLE PROMPT INSTRUCTIONS")
    for prompt in PromptCatalog().get_all_prompts():
        print (" - " + prompt["prompt_name"])

# With the provided context submit the given prompt to the LLM
def simple_prompt_with_context_string(prompt, context, llm_name, api_key):
    print (f"\n > SIMPLE PROMPT WITH CONTEXT STRING")
    prompter = Prompt(llm_name=llm_name, llm_api_key=api_key)
    response = prompter.prompt_main(prompt=prompt, context=context)["llm_response"]
    print (f"- Context: {context}\n- Prompt: {prompt}\n- LLM Response:\n{response}")

# Use an llmware prompt_instruction to submit the given prompt and prompt_instruction to the LLM
def prompt_with_prompt_instruction(prompt, context, prompt_instruction, llm_name, api_key):
    print (f"\n > PROMPT WITH CONTEXT USING '{prompt_instruction}' PROMPT INSTRUCTION")
    prompter = Prompt(llm_name=llm_name, llm_api_key=api_key)
    response = prompter.prompt_from_catalog(prompt=prompt, context=context, prompt_name=prompt_instruction)["llm_response"]
    print (f"- Context: {context}\n- Prompt: {prompt}\n- LLM Response:\n{response}")


# In some cases you may want to add additional configuraton.
def prompt_with_inference_config(prompt, context, prompt_instruction, inference_config, llm_name, api_key):
    print (f"\n > PROMPT WITH CONTEXT USING '{prompt_instruction}' PROMPT INSTRUCTION")
    prompter = Prompt(llm_name=llm_name, llm_api_key=api_key)
    response = prompter.prompt_main(prompt=prompt, context=context, prompt_name=prompt_instruction, 
                                    inference_dict=inference_config)["llm_response"]
    print (f"- Context: {context}\n- Prompt: {prompt}\n- LLM Response:\n{response}")

# If the context you need to pass to an LLM is contained in Wikipedia you can easily add it as a source
def prompt_with_wiki_source(prompt, wiki_topic, prompt_instruction, llm_name, api_key):
    print (f"\n > PROMPT WITH CONTEXT FROM WIKIPEDIA USING '{prompt_instruction}' PROMPT INSTRUCTION")
    prompter = Prompt(llm_name=llm_name, llm_api_key=api_key)
    prompter.add_source_wikipedia(wiki_topic, article_count=1)
    response = prompter.prompt_with_source(prompt=prompt, prompt_name=prompt_instruction)[0]["llm_response"]
    print (f"- Context: Wikepedia article(s) for '{wiki_topic}'\n- Prompt: {prompt}\n- LLM Response:\n{response}")

# If the context you need to pass is in local files, you can easily add then as sources
def prompt_with_local_file_sources(prompt, local_folder, local_files, prompt_instruction, llm_name, api_key):
    print (f"\n > PROMPT WITH CONTEXT FROM LOCAL FILE USING '{prompt_instruction}' PROMPT INSTRUCTION")
    prompter = Prompt(llm_name=llm_name, llm_api_key=api_key)
    for local_file in local_files:
        prompter.add_source_document(local_folder, local_file)
    response = prompter.prompt_with_source(prompt=prompt, prompt_name=prompt_instruction)[0]["llm_response"]
    print (f"- Context: {local_files}\n- Prompt: {prompt}\n- LLM Response:\n{response}")

In [6]:
print_all_prompt_instructions()


 > ALL AVAILABLE PROMPT INSTRUCTIONS
 - just_the_facts
 - answer_or_not_found
 - number_or_none
 - summarize_with_bullets
 - summarize_with_numbered_bullets
 - xsummary
 - completion
 - dialog_summary
 - not_found_classifier
 - top_level_select
 - answer_question_in_role
 - editor_in_role
 - yes_no
 - multiple_choice
 - default_with_context
 - default_no_context
 - summarize_with_bullets_w_query
 - summarize_with_references_w_query
 - write_poem
 - ten_words
 - explain_child
 - make_joke
 - tell_story
 - write_headline
 - facts_only
 - top_bulletpoints
 - report_title
 - marketing_slogan
 - top_level_summary


In [7]:
simple_prompt_with_context_string( prompt = "What is my 3rd favorite type of food?",
                                   context = "My favorite foods are Sushi, Italian and Greek",
                                   llm_name = OPENAI_MODEL_NAME,
                                   api_key = openai_api_key
                                 )


 > SIMPLE PROMPT WITH CONTEXT STRING
- Context: My favorite foods are Sushi, Italian and Greek
- Prompt: What is my 3rd favorite type of food?
- LLM Response:
Your 3rd favorite type of food is Greek.


In [8]:
prompt_with_prompt_instruction( prompt = "How old is my oldest sibling?",
                                context = "My brother is 20 years old and my sister is 1.5 times older",
                                prompt_instruction = "number_or_none",
                                llm_name = OPENAI_MODEL_NAME,
                                api_key = openai_api_key
                              )


 > PROMPT WITH CONTEXT USING 'number_or_none' PROMPT INSTRUCTION
- Context: My brother is 20 years old and my sister is 1.5 times older
- Prompt: How old is my oldest sibling?
- LLM Response:
Your oldest sibling's age is not found in the given text.


In [9]:
prompt_with_inference_config( prompt = "Why is it difficult?",
                              context = "I am interested in building rockets",
                              prompt_instruction = "explain_child",
                              inference_config = {"temperature": 0.8, "llm_max_output_len": 1000, "max_tokens": 1000},
                              llm_name = OPENAI_MODEL_NAME,
                              api_key = openai_api_key)


 > PROMPT WITH CONTEXT USING 'explain_child' PROMPT INSTRUCTION
- Context: I am interested in building rockets
- Prompt: Why is it difficult?
- LLM Response:
Building rockets is difficult because it requires a lot of complex and advanced knowledge. You need to understand things like physics and engineering to be able to design and build a rocket. It also takes a lot of time and effort to gather all the materials needed and put them together in the right way. There are also many safety precautions that need to be taken into consideration. So, while building rockets can be really exciting and fun, it is also a challenging task that requires a lot of skill and hard work.


In [10]:
prompt_with_wiki_source( prompt = "Was Barack Obama the Prime Minister of Canada?",
                         wiki_topic = "Barack Obama",
                         prompt_instruction = "yes_no",
                         llm_name = OPENAI_MODEL_NAME,
                         api_key = openai_api_key)


 > PROMPT WITH CONTEXT FROM WIKIPEDIA USING 'yes_no' PROMPT INSTRUCTION
- Context: Wikepedia article(s) for 'Barack Obama'
- Prompt: Was Barack Obama the Prime Minister of Canada?
- LLM Response:
No


In [11]:
prompt_with_local_file_sources( prompt = "What was the effective date of this agreement?",
                                local_folder = os.path.join(Setup().load_sample_files(), "SmallLibrary"),
                                local_files = ['Gaia EXECUTIVE EMPLOYMENT AGREEMENT.pdf'],
                                prompt_instruction = "just_the_facts",
                                llm_name = OPENAI_MODEL_NAME,
                                api_key = openai_api_key)


 > PROMPT WITH CONTEXT FROM LOCAL FILE USING 'just_the_facts' PROMPT INSTRUCTION
- Context: ['Gaia EXECUTIVE EMPLOYMENT AGREEMENT.pdf']
- Prompt: What was the effective date of this agreement?
- LLM Response:
The effective date of this agreement is May 1, 2015.
